<a href="https://colab.research.google.com/github/velumahalingam/AutoOED/blob/master/Copy_of_Capacity_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from collections import defaultdict
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
import os

warnings.filterwarnings('ignore')

class CapacityPlanningOptimizer:
    def __init__(self):
        # Suite configurations in priority order
        self.suite_priority_order = [
            'PW-S1', 'PW-S2', 'Pipan-S1', 'Pipan-S2', 'PW-S3', 'Pipan-S6', 'Pipan-S7',
            'Pipan-S3', 'Pipan-S4', 'Pipan-S5', 'Pipan-S8'
        ]

        self.suite_config = {
            'PW-S1': {'process': 'Fed-Batch', 'reactors': 1, 'scale_L': 500},
            'PW-S2': {'process': 'Fed-Batch', 'reactors': 3, 'scale_L': 2000},
            'PW-S3': {'process': 'Perfusion', 'reactors': 2, 'scale_L': 1000},
            'Pipan-S1': {'process': 'Fed-Batch', 'reactors': 2, 'scale_L': 2000},
            'Pipan-S2': {'process': 'Perfusion', 'reactors': 2, 'scale_L': 1000},
            'Pipan-S3': {'process': 'Fed-Batch', 'reactors': 2, 'scale_L': 2000},
            'Pipan-S4': {'process': 'Fed-Batch', 'reactors': 2, 'scale_L': 2000},
            'Pipan-S5': {'process': 'Fed-Batch', 'reactors': 2, 'scale_L': 2000},
            'Pipan-S6': {'process': 'Fed-Batch', 'reactors': 2, 'scale_L': 5000},
            'Pipan-S7': {'process': 'Fed-Batch', 'reactors': 2, 'scale_L': 5000},
            'Pipan-S8': {'process': 'Fed-Batch', 'reactors': 2, 'scale_L': 2000}
        }

        # Product specifications
        self.product_specs = {
            'ADL-018': {'productivity_g_L': 1.4, 'process': 'Fed-Batch'},
            'KSHB002': {'productivity_g_L': 0.7, 'process': 'Fed-Batch'},
            'KSHB005': {'productivity_g_L': 1.2, 'process': 'Fed-Batch'},
            'KSHB006': {'productivity_g_L': 5.0, 'process': 'Perfusion'},
            'KSHB010': {'productivity_g_L': 3, 'process': 'Fed-Batch'},
            'KSHB012': {'productivity_g_L': 3, 'process': 'Fed-Batch'},
            'KSHB013': {'productivity_g_L': 3, 'process': 'Fed-Batch'},
            'KSHB015': {'productivity_g_L': 3, 'process': 'Fed-Batch'},
            'KSHB017': {'productivity_g_L': 3, 'process': 'Fed-Batch'},
            'KSHB018': {'productivity_g_L': 3, 'process': 'Fed-Batch'},
            'KSHB019': {'productivity_g_L': 3, 'process': 'Fed-Batch'},
            'KSHB020': {'productivity_g_L': 3, 'process': 'Fed-Batch'}
        }

        # Process constraints
        self.max_batches_per_year = {
            'Perfusion': 11,
            'Fed-Batch': 22
        }

        # Default productivity for unlisted products
        self.default_productivity = 2.5
        self.default_process = 'Fed-Batch'

        # Enhanced tracking for suite utilization per year
        self.suite_utilization = {}
        self.suite_allocations = {}
        self.batch_usage = {}

    def load_demand_data(self, file_path):
        """Load demand data from Excel file"""
        try:
            df = pd.read_excel(file_path)

            # Check if data has the Product/Label format
            if 'Label' in df.columns:
                # Filter only demand rows
                demand_df = df[df['Label'].str.contains('Demand', na=False)].copy()
                # Remove the Label column
                demand_df = demand_df.drop('Label', axis=1)
            else:
                demand_df = df.copy()

            # Ensure Product column exists
            if 'Product' not in demand_df.columns:
                demand_df = demand_df.rename(columns={demand_df.columns[0]: 'Product'})

            return demand_df
        except Exception as e:
            print(f"Error loading demand data: {e}")
            return None

    def get_product_specs(self, product):
        """Get product specifications with defaults for unlisted products"""
        if product in self.product_specs:
            return self.product_specs[product]
        else:
            return {
                'productivity_g_L': self.default_productivity,
                'process': self.default_process
            }

    def calculate_suite_max_capacity(self, suite_name, product):
        """Calculate maximum annual production capacity for a suite-product combination"""
        suite = self.suite_config[suite_name]
        product_spec = self.get_product_specs(product)

        # Check process compatibility
        if suite['process'] != product_spec['process']:
            return 0

        # Calculate maximum capacity for the entire suite
        max_batches = self.max_batches_per_year[suite['process']]
        productivity = product_spec['productivity_g_L']
        scale = suite['scale_L']
        total_reactors = suite['reactors']

        # Total annual capacity = batches/year × scale × productivity × total_reactors
        total_annual_capacity_g = max_batches * scale * productivity * total_reactors
        total_annual_capacity_kg = total_annual_capacity_g / 1000

        return total_annual_capacity_kg

    def calculate_batches_needed(self, suite_name, product, required_kg):
        """Calculate number of batches needed for a given production requirement"""
        suite = self.suite_config[suite_name]
        product_spec = self.get_product_specs(product)

        if suite['process'] != product_spec['process']:
            return 0

        productivity = product_spec['productivity_g_L']
        scale = suite['scale_L']
        total_reactors = suite['reactors']

        # Convert kg to g
        required_g = required_kg * 1000

        # Calculate batches needed
        batch_capacity_g = scale * productivity * total_reactors
        batches_needed = np.ceil(required_g / batch_capacity_g)

        return int(batches_needed)

    def initialize_suite_tracking(self, years):
        """Initialize comprehensive suite tracking for each year"""
        for year in years:
            # Initialize suite utilization tracking
            self.suite_utilization[year] = {}
            self.suite_allocations[year] = {}
            self.batch_usage[year] = {}

            for suite in self.suite_config.keys():
                self.suite_utilization[year][suite] = {
                    'max_capacity_per_product': {},
                    'used_capacity': 0,
                    'allocations': [],
                    'batches_used': 0,
                    'max_batches': self.max_batches_per_year[self.suite_config[suite]['process']]
                }

                self.suite_allocations[year][suite] = {}
                self.batch_usage[year][suite] = {}

                # Pre-calculate max capacity for each product
                for product in self.product_specs.keys():
                    max_cap = self.calculate_suite_max_capacity(suite, product)
                    if max_cap > 0:
                        self.suite_utilization[year][suite]['max_capacity_per_product'][product] = max_cap

    def check_batch_availability(self, suite, year, product, required_kg):
        """Check if suite has enough batch capacity for the required production"""
        batches_needed = self.calculate_batches_needed(suite, product, required_kg)
        batches_used = self.suite_utilization[year][suite]['batches_used']
        max_batches = self.suite_utilization[year][suite]['max_batches']

        available_batches = max_batches - batches_used

        return batches_needed <= available_batches, batches_needed, available_batches

    def allocate_capacity(self, suite, year, product, requested_kg):
        """Enhanced allocation with batch tracking and double allocation prevention"""
        # Check if we can fulfill this request
        can_fulfill, batches_needed, available_batches = self.check_batch_availability(
            suite, year, product, requested_kg
        )

        if not can_fulfill:
            # Calculate what we can actually produce with available batches
            suite_config = self.suite_config[suite]
            product_spec = self.get_product_specs(product)

            if available_batches <= 0:
                return 0

            batch_capacity_kg = (suite_config['scale_L'] * product_spec['productivity_g_L'] *
                               suite_config['reactors']) / 1000

            max_possible_kg = available_batches * batch_capacity_kg
            allocated = min(requested_kg, max_possible_kg)
            batches_needed = available_batches
        else:
            allocated = requested_kg

        if allocated > 0:
            # Update batch usage
            self.suite_utilization[year][suite]['batches_used'] += batches_needed

            # Update capacity usage
            self.suite_utilization[year][suite]['used_capacity'] += allocated

            # Track allocation by product
            if product not in self.suite_allocations[year][suite]:
                self.suite_allocations[year][suite][product] = 0
            self.suite_allocations[year][suite][product] += allocated

            # Track batch usage by product
            if product not in self.batch_usage[year][suite]:
                self.batch_usage[year][suite][product] = 0
            self.batch_usage[year][suite][product] += batches_needed

            # Calculate utilization percentage
            max_capacity = self.suite_utilization[year][suite]['max_capacity_per_product'].get(product, 0)
            utilization_pct = (allocated / max_capacity * 100) if max_capacity > 0 else 0

            # Record allocation
            self.suite_utilization[year][suite]['allocations'].append({
                'product': product,
                'allocated_kg': allocated,
                'batches_used': batches_needed,
                'utilization_pct': utilization_pct
            })

            return allocated

        return 0

    def optimize_allocation_by_year_shared_capacity(self, demand_df):
        """Enhanced optimization with strict batch tracking"""
        # Convert demand to long format
        product_col = 'Product'
        year_cols = [col for col in demand_df.columns if isinstance(col, (int, float)) and col >= 2020]

        demand_long = pd.melt(demand_df,
                            id_vars=[product_col],
                            value_vars=year_cols,
                            var_name='Year',
                            value_name='Demand')

        # Clean demand data
        demand_long = demand_long.dropna(subset=['Demand'])
        demand_long = demand_long[demand_long['Demand'] > 0].copy()
        demand_long = demand_long.sort_values(['Year', 'Product'])

        # Initialize suite tracking
        years = sorted(year_cols)
        self.initialize_suite_tracking(years)

        allocation_results = []

        # Process each year separately
        for year in years:
            year_demand = demand_long[demand_long['Year'] == year].copy()
            print(f"\nProcessing Year {year} with {len(year_demand)} products")

            # Create a list of all product demands for this year
            product_demands = []
            for _, demand_row in year_demand.iterrows():
                product = demand_row['Product']
                required_kg = float(demand_row['Demand'])
                product_spec = self.get_product_specs(product)

                product_demands.append({
                    'product': product,
                    'demand_kg': required_kg,
                    'remaining_kg': required_kg,
                    'process': product_spec['process']
                })

            # Sort by demand (highest first) to prioritize high-demand products
            product_demands.sort(key=lambda x: x['demand_kg'], reverse=True)

            # Process suites in priority order
            for suite_name in self.suite_priority_order:
                suite_config = self.suite_config[suite_name]

                print(f"  Processing Suite: {suite_name} ({suite_config['process']}) - Batches available: {self.suite_utilization[year][suite_name]['max_batches'] - self.suite_utilization[year][suite_name]['batches_used']}")

                # Find products compatible with this suite that still have unmet demand
                compatible_products = [
                    p for p in product_demands
                    if p['process'] == suite_config['process'] and p['remaining_kg'] > 0
                ]

                if not compatible_products:
                    continue

                # Allocate capacity to products in order of demand priority
                for product_info in compatible_products:
                    if product_info['remaining_kg'] <= 0:
                        continue

                    product = product_info['product']
                    requested_kg = product_info['remaining_kg']

                    # Check batch availability before allocation
                    can_fulfill, batches_needed, available_batches = self.check_batch_availability(
                        suite_name, year, product, requested_kg
                    )

                    if available_batches <= 0:
                        continue

                    # Try to allocate capacity
                    allocated_kg = self.allocate_capacity(suite_name, year, product, requested_kg)

                    if allocated_kg > 0:
                        # Calculate metrics
                        max_capacity = self.suite_utilization[year][suite_name]['max_capacity_per_product'].get(product, 0)
                        utilization_pct = (allocated_kg / max_capacity * 100) if max_capacity > 0 else 0

                        # Calculate total suite utilization based on batch usage
                        batch_utilization = (self.suite_utilization[year][suite_name]['batches_used'] /
                                           self.suite_utilization[year][suite_name]['max_batches'] * 100)

                        allocation_results.append({
                            'Product': product,
                            'Year': year,
                            'Suite': suite_name,
                            'Max_Suite_Capacity_kg': max_capacity,
                            'Allocated_kg': allocated_kg,
                            'Batches_Used': self.batch_usage[year][suite_name].get(product, 0),
                            'Product_Utilization_%': utilization_pct,
                            'Suite_Batch_Utilization_%': batch_utilization,
                            'Process': product_info['process'],
                            'Scale_L': suite_config['scale_L'],
                            'Reactors': suite_config['reactors'],
                            'Productivity_g_L': self.get_product_specs(product)['productivity_g_L']
                        })

                        # Update remaining demand
                        product_info['remaining_kg'] -= allocated_kg

                        print(f"    {product}: Allocated {allocated_kg:.1f} kg ({self.batch_usage[year][suite_name].get(product, 0)} batches, {utilization_pct:.1f}% product util)")

            # Track unmet demand
            for product_info in product_demands:
                if product_info['remaining_kg'] > 0:
                    allocation_results.append({
                        'Product': product_info['product'],
                        'Year': year,
                        'Suite': 'UNMET_DEMAND',
                        'Max_Suite_Capacity_kg': 0,
                        'Allocated_kg': product_info['remaining_kg'],
                        'Batches_Used': 0,
                        'Product_Utilization_%': 0,
                        'Suite_Batch_Utilization_%': 0,
                        'Process': product_info['process'],
                        'Scale_L': 0,
                        'Reactors': 0,
                        'Productivity_g_L': 0
                    })
                    print(f"    UNMET DEMAND - {product_info['product']}: {product_info['remaining_kg']:.1f} kg")

        return pd.DataFrame(allocation_results)

    def generate_comprehensive_report(self, allocation_df):
        """Generate comprehensive analysis reports"""
        reports = {}

        # 1. Yearly Summary Report
        yearly_summary = allocation_df.groupby('Year').agg({
            'Allocated_kg': 'sum'
        }).reset_index()

        # Calculate unmet demand by year
        unmet_by_year = allocation_df[allocation_df['Suite'] == 'UNMET_DEMAND'].groupby('Year')['Allocated_kg'].sum()
        yearly_summary = yearly_summary.merge(
            unmet_by_year.reset_index().rename(columns={'Allocated_kg': 'Unmet_Demand_kg'}),
            on='Year', how='left'
        )
        yearly_summary['Unmet_Demand_kg'] = yearly_summary['Unmet_Demand_kg'].fillna(0)
        yearly_summary['Total_Demand_kg'] = yearly_summary['Allocated_kg']
        yearly_summary['Fulfilled_kg'] = yearly_summary['Allocated_kg'] - yearly_summary['Unmet_Demand_kg']
        yearly_summary['Fulfillment_%'] = (yearly_summary['Fulfilled_kg'] / yearly_summary['Total_Demand_kg'] * 100).round(2)
        yearly_summary = yearly_summary.set_index('Year')

        reports['yearly_summary'] = yearly_summary

        # 2. Suite Utilization Report
        suite_util = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND'].groupby(['Suite', 'Year']).agg({
            'Suite_Batch_Utilization_%': 'max',
            'Allocated_kg': 'sum',
            'Max_Suite_Capacity_kg': 'first'
        }).reset_index()

        reports['suite_utilization'] = suite_util

        # 3. Product Analysis Report
        product_analysis = allocation_df.groupby('Product').agg({
            'Allocated_kg': 'sum',
            'Process': 'first',
            'Productivity_g_L': 'first'
        }).reset_index()

        # Calculate unmet demand by product
        unmet_by_product = allocation_df[allocation_df['Suite'] == 'UNMET_DEMAND'].groupby('Product')['Allocated_kg'].sum()
        product_analysis = product_analysis.merge(
            unmet_by_product.reset_index().rename(columns={'Allocated_kg': 'Unmet_Demand_kg'}),
            on='Product', how='left'
        )
        product_analysis['Unmet_Demand_kg'] = product_analysis['Unmet_Demand_kg'].fillna(0)
        product_analysis['Total_Demand_kg'] = product_analysis['Allocated_kg']
        product_analysis['Fulfilled_kg'] = product_analysis['Allocated_kg'] - product_analysis['Unmet_Demand_kg']
        product_analysis['Fulfillment_%'] = (product_analysis['Fulfilled_kg'] / product_analysis['Total_Demand_kg'] * 100).round(2)

        reports['product_analysis'] = product_analysis

        # 4. Capacity Gaps Report
        capacity_gaps = []
        for year in allocation_df['Year'].unique():
            year_data = allocation_df[allocation_df['Year'] == year]
            unmet = year_data[year_data['Suite'] == 'UNMET_DEMAND']

            if not unmet.empty:
                for _, row in unmet.iterrows():
                    capacity_gaps.append({
                        'Year': year,
                        'Product': row['Product'],
                        'Unmet_Demand_kg': row['Allocated_kg'],
                        'Process': row['Process'],
                        'Recommendation': f"Additional {row['Process']} capacity needed"
                    })

        reports['capacity_gaps'] = pd.DataFrame(capacity_gaps)

        return reports
    def create_demand_vs_allocation_table(self, results):
        """Create detailed demand vs allocation table by product and suite"""
        allocation_df = results['allocation']

        # Get original demand data for comparison
        demand_pivot = allocation_df.groupby(['Product', 'Year']).agg({
            'Allocated_kg': 'sum'  # This includes unmet demand
        }).reset_index()

        # Get allocation by suite
        suite_allocation = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND'].pivot_table(
            index ['Product', 'Year'],
            columns='Suite',
            values='Allocated_kg',
            fill_value=0
        ).reset_index()

        # Get unmet demand
        unmet_demand = allocation_df[allocation_df['Suite'] == 'UNMET_DEMAND'].pivot_table(
            index=['Product', 'Year'],
            values='Allocated_kg',
            fill_value=0
        ).reset_index()
        unmet_demand = unmet_demand.rename(columns={'Allocated_kg': 'Unmet_Demand_kg'})

        # Merge all data
        demand_vs_allocation = demand_pivot.merge(suite_allocation, on=['Product', 'Year'], how='left')
        demand_vs_allocation = demand_vs_allocation.merge(unmet_demand, on=['Product', 'Year'], how='left')
        demand_vs_allocation['Unmet_Demand_kg'] = demand_vs_allocation['Unmet_Demand_kg'].fillna(0)

        # Calculate total allocated (excluding unmet)
        suite_cols = [col for col in demand_vs_allocation.columns if col.startswith(('PW-', 'Pipan-'))]
        demand_vs_allocation['Total_Allocated_kg'] = demand_vs_allocation[suite_cols].sum(axis=1)
        demand_vs_allocation['Total_Demand_kg'] = demand_vs_allocation['Allocated_kg']
        demand_vs_allocation['Fulfillment_%'] = (demand_vs_allocation['Total_Allocated_kg'] / demand_vs_allocation['Total_Demand_kg'] * 100).round(2)

        # Reorder columns
        base_cols = ['Product', 'Year', 'Total_Demand_kg', 'Total_Allocated_kg', 'Fulfillment_%']
        suite_cols = sorted(suite_cols)
        other_cols = ['Unmet_Demand_kg']

        final_cols = base_cols + suite_cols + other_cols
        demand_vs_allocation = demand_vs_allocation[final_cols].fillna(0)

        return demand_vs_allocation


    def create_visualizations(self, results, output_dir='capacity_planning_charts'):
        """Create comprehensive visualizations"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Set style
        plt.style.use('default')

        # 1. Suite Utilization Heatmap
        self.create_suite_utilization_heatmap(results, output_dir)

        # 2. Demand vs Capacity Over Time
        self.create_demand_capacity_timeline(results, output_dir)

        # 3. Product Allocation Breakdown
        self.create_product_allocation_breakdown(results, output_dir)

        # 4. Suite Capacity Waterfall Chart
        self.create_suite_capacity_waterfall(results, output_dir)

        # 5. Batch Utilization Analysis
        self.create_batch_utilization_analysis(results, output_dir)

        # 6. Interactive Plotly Dashboard
        self.create_interactive_dashboard(results, output_dir)

        print(f"\n📊 All visualizations saved to: {output_dir}/")

    def create_suite_utilization_heatmap(self, results, output_dir):
        """Create suite utilization heatmap"""
        # Prepare data for heatmap
        allocation_df = results['allocation']

        # Create utilization matrix
        utilization_data = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND'].pivot_table(
            index='Suite',
            columns='Year',
            values='Suite_Batch_Utilization_%',
            aggfunc='max',
            fill_value=0
        )

        # Create heatmap
        plt.figure(figsize=(14, 8))
        sns.heatmap(utilization_data,
                   annot=True,
                   fmt='.1f',
                   cmap='RdYlBu_r',
                   center=50,
                   cbar_kws={'label': 'Utilization %'})

        plt.title('Suite Utilization Heatmap Over Years', fontsize=16, fontweight='bold')
        plt.xlabel('Year', fontsize=12)
        plt.ylabel('Manufacturing Suite', fontsize=12)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/suite_utilization_heatmap.png', dpi=300, bbox_inches='tight')
        plt.close()

    def create_demand_capacity_timeline(self, results, output_dir):
        """Create demand vs capacity timeline"""
        yearly_summary = results['reports']['yearly_summary']

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

        # Top chart - Demand vs Allocated
        years = yearly_summary.index
        ax1.bar(years, yearly_summary['Total_Demand_kg'],
               alpha=0.7, label='Total Demand', color='lightcoral')
        ax1.bar(years, yearly_summary['Allocated_kg'] - yearly_summary['Unmet_Demand_kg'],
               alpha=0.8, label='Allocated', color='lightblue')
        ax1.bar(years, yearly_summary['Unmet_Demand_kg'],
               bottom=yearly_summary['Allocated_kg'] - yearly_summary['Unmet_Demand_kg'],
               alpha=0.8, label='Unmet Demand', color='red')

        ax1.set_title('Annual Demand vs Production Capacity', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Production (kg)', fontsize=12)
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Bottom chart - Fulfillment percentage
        ax2.plot(years, yearly_summary['Fulfillment_%'],
                marker='o', linewidth=3, markersize=8, color='green')
        ax2.fill_between(years, yearly_summary['Fulfillment_%'],
                        alpha=0.3, color='green')
        ax2.axhline(y=100, color='red', linestyle='--', alpha=0.7, label='100% Fulfillment')

        ax2.set_title('Demand Fulfillment Rate Over Time', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Year', fontsize=12)
        ax2.set_ylabel('Fulfillment %', fontsize=12)
        ax2.set_ylim(0, 105)
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(f'{output_dir}/demand_capacity_timeline.png', dpi=300, bbox_inches='tight')
        plt.close()

    def create_product_allocation_breakdown(self, results, output_dir):
        """Create product allocation breakdown"""
        allocation_df = results['allocation']

        # Product allocation by suite
        product_suite = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND'].groupby(['Product', 'Suite'])['Allocated_kg'].sum().reset_index()

        # Create subplots for each product
        products = product_suite['Product'].unique()
        n_products = len(products)

        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()

        for i, product in enumerate(products[:6]):  # Show first 6 products
            if i < len(axes):
                product_data = product_suite[product_suite['Product'] == product]

                if not product_data.empty:
                    wedges, texts, autotexts = axes[i].pie(
                        product_data['Allocated_kg'],
                        labels=product_data['Suite'],
                        autopct='%1.1f%%',
                        startangle=90
                    )
                    axes[i].set_title(f'{product} - Suite Allocation', fontweight='bold')
                else:
                    axes[i].text(0.5, 0.5, 'No Allocation', ha='center', va='center', transform=axes[i].transAxes)
                    axes[i].set_title(f'{product} - No Allocation', fontweight='bold')

        # Hide unused subplots
        for i in range(len(products), len(axes)):
            axes[i].set_visible(False)

        plt.suptitle('Product Allocation by Manufacturing Suite', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'{output_dir}/product_allocation_breakdown.png', dpi=300, bbox_inches='tight')
        plt.close()

    def create_suite_capacity_waterfall(self, results, output_dir):
        """Create suite capacity waterfall chart"""
        allocation_df = results['allocation']

        # Calculate total capacity and usage by suite
        suite_summary = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND'].groupby('Suite').agg({
            'Max_Suite_Capacity_kg': 'first',
            'Allocated_kg': 'sum'
        }).reset_index()

        suite_summary['Available_Capacity'] = suite_summary['Max_Suite_Capacity_kg'] - suite_summary['Allocated_kg']
        suite_summary = suite_summary.sort_values('Max_Suite_Capacity_kg', ascending=False)

        # Create stacked bar chart
        fig, ax = plt.subplots(figsize=(14, 8))

        x_pos = range(len(suite_summary))

        # Used capacity
        ax.bar(x_pos, suite_summary['Allocated_kg'],
               label='Used Capacity', color='lightcoral', alpha=0.8)

        # Available capacity
        ax.bar(x_pos, suite_summary['Available_Capacity'],
               bottom=suite_summary['Allocated_kg'],
               label='Available Capacity', color='lightblue', alpha=0.8)

        ax.set_xlabel('Manufacturing Suite', fontsize=12)
        ax.set_ylabel('Capacity (kg)', fontsize=12)
        ax.set_title('Suite Capacity Utilization Overview', fontsize=14, fontweight='bold')
        ax.set_xticks(x_pos)
        ax.set_xticklabels(suite_summary['Suite'], rotation=45, ha='right')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Add utilization percentages
        for i, (idx, row) in enumerate(suite_summary.iterrows()):
            utilization = (row['Allocated_kg'] / row['Max_Suite_Capacity_kg'] * 100) if row['Max_Suite_Capacity_kg'] > 0 else 0
            ax.text(i, row['Max_Suite_Capacity_kg'] + 50, f'{utilization:.1f}%',
                   ha='center', va='bottom', fontweight='bold')

        plt.tight_layout()
        plt.savefig(f'{output_dir}/suite_capacity_waterfall.png', dpi=300, bbox_inches='tight')
        plt.close()


        # Batch utilization by suite and year
    def create_batch_utilization_analysis(self, results, output_dir):
        """Create batch utilization analysis"""
        allocation_df = results['allocation']

        # Batch utilization by suite and year
        batch_data = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND'].copy()

        # Calculate batch utilization by suite and process type
        process_utilization = batch_data.groupby(['Process', 'Year']).agg({
            'Batches_Used': 'sum',
            'Suite_Batch_Utilization_%': 'mean'
        }).reset_index()

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

        # Batch usage by process type
        fed_batch_data = process_utilization[process_utilization['Process'] == 'Fed-Batch']
        perfusion_data = process_utilization[process_utilization['Process'] == 'Perfusion']

        if not fed_batch_data.empty:
            ax1.plot(fed_batch_data['Year'], fed_batch_data['Batches_Used'],
                    marker='o', linewidth=2, label='Fed-Batch', color='blue')

        if not perfusion_data.empty:
            ax1.plot(perfusion_data['Year'], perfusion_data['Batches_Used'],
                    marker='s', linewidth=2, label='Perfusion', color='red')

        ax1.set_title('Batch Usage by Process Type', fontweight='bold')
        ax1.set_xlabel('Year')
        ax1.set_ylabel('Total Batches Used')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Average utilization by process type
        if not fed_batch_data.empty:
            ax2.plot(fed_batch_data['Year'], fed_batch_data['Suite_Batch_Utilization_%'],
                    marker='o', linewidth=2, label='Fed-Batch', color='blue')

        if not perfusion_data.empty:
            ax2.plot(perfusion_data['Year'], perfusion_data['Suite_Batch_Utilization_%'],
                    marker='s', linewidth=2, label='Perfusion', color='red')

        ax2.set_title('Average Batch Utilization by Process Type', fontweight='bold')
        ax2.set_xlabel('Year')
        ax2.set_ylabel('Average Utilization %')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(f'{output_dir}/batch_utilization_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()

    def create_demand_allocation_plot(self, results, output_dir):
        """Create demand vs allocation visualization by product"""
        demand_allocation_df = self.create_demand_vs_allocation_table(results)

        # Get top 6 products by total demand
        top_products = demand_allocation_df.groupby('Product')['Total_Demand_kg'].sum().nlargest(6).index

        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        axes = axes.flatten()

        suite_cols = [col for col in demand_allocation_df.columns if col.startswith(('PW-', 'Pipan-'))]
        colors = plt.cm.Set3(np.linspace(0, 1, len(suite_cols)))

        for i, product in enumerate(top_products):
            if i < len(axes):
                product_data = demand_allocation_df[demand_allocation_df['Product'] == product]

                # Create stacked bar for each year
                bottom = np.zeros(len(product_data))

                for j, suite in enumerate(suite_cols):
                    if suite in product_data.columns and product_data[suite].sum() > 0:
                        axes[i].bar(product_data['Year'], product_data[suite],
                                  bottom=bottom, label=suite, color=colors[j], alpha=0.8)
                        bottom += product_data[suite]

                # Add unmet demand on top
                if product_data['Unmet_Demand_kg'].sum() > 0:
                    axes[i].bar(product_data['Year'], product_data['Unmet_Demand_kg'],
                              bottom=bottom, label='Unmet Demand', color='red', alpha=0.8)

                # Add total demand line
                axes[i].plot(product_data['Year'], product_data['Total_Demand_kg'],
                            'ko-', linewidth=2, markersize=4, label='Total Demand')

                axes[i].set_title(f'{product} - Demand vs Allocation by Suite', fontweight='bold')
                axes[i].set_xlabel('Year')
                axes[i].set_ylabel('Quantity (kg)')
                axes[i].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                axes[i].grid(True, alpha=0.3)

        # Hide unused subplots
        for i in range(len(top_products), len(axes)):
            axes[i].set_visible(False)

        plt.suptitle('Product Demand vs Suite Allocation Over Time', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'{output_dir}/demand_vs_allocation_by_product.png', dpi=300, bbox_inches='tight')
        plt.close()
    def create_interactive_dashboard(self, results, output_dir):
        """Create interactive Plotly dashboard"""
        allocation_df = results['allocation']

        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Suite Utilization Over Time', 'Product Demand Timeline',
                          'Capacity vs Demand by Year', 'Process Type Distribution'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": True}, {"type": "pie"}]]
        )

        # 1. Suite Utilization Over Time
        suite_util = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND']
        for suite in suite_util['Suite'].unique():
            suite_data = suite_util[suite_util['Suite'] == suite]
            fig.add_trace(
                go.Scatter(x=suite_data['Year'],
                          y=suite_data['Suite_Batch_Utilization_%'],
                          mode='lines+markers',
                          name=suite,
                          showlegend=True),
                row=1, col=1
            )

        # 2. Product Demand Timeline
        product_demand = allocation_df.groupby(['Year', 'Product'])['Allocated_kg'].sum().reset_index()
        top_products = product_demand.groupby('Product')['Allocated_kg'].sum().nlargest(5).index

        for product in top_products:
            product_data = product_demand[product_demand['Product'] == product]
            fig.add_trace(
                go.Scatter(x=product_data['Year'],
                          y=product_data['Allocated_kg'],
                          mode='lines+markers',
                          name=product,
                          showlegend=False),
                row=1, col=2
            )

        # 3. Capacity vs Demand by Year
        yearly_summary = results['reports']['yearly_summary']
        fig.add_trace(
            go.Bar(x=yearly_summary.index,
                   y=yearly_summary['Fulfilled_kg'],
                   name='Fulfilled Demand',
                   marker_color='lightblue'),
            row=2, col=1
        )

        fig.add_trace(
            go.Bar(x=yearly_summary.index,
                   y=yearly_summary['Unmet_Demand_kg'],
                   name='Unmet Demand',
                   marker_color='red'),
            row=2, col=1
        )

        # 4. Process Type Distribution
        process_dist = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND'].groupby('Process')['Allocated_kg'].sum()
        fig.add_trace(
            go.Pie(labels=process_dist.index,
                   values=process_dist.values,
                   name="Process Distribution"),
            row=2, col=2
        )

        # Update layout
        fig.update_layout(
            height=800,
            title_text="Capacity Planning Dashboard",
            title_x=0.5,
            showlegend=True
        )

        # Update axes labels
        fig.update_xaxes(title_text="Year", row=1, col=1)
        fig.update_yaxes(title_text="Utilization %", row=1, col=1)
        fig.update_xaxes(title_text="Year", row=1, col=2)
        fig.update_yaxes(title_text="Demand (kg)", row=1, col=2)
        fig.update_xaxes(title_text="Year", row=2, col=1)
        fig.update_yaxes(title_text="Production (kg)", row=2, col=1)

        # Save interactive plot
        fig.write_html(f'{output_dir}/interactive_dashboard.html')

        print(f"📊 Interactive dashboard saved: {output_dir}/interactive_dashboard.html")

    def export_results_to_excel(self, results, filename='capacity_planning_results.xlsx'):
        """Export all results to Excel with multiple sheets"""
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # Main allocation results
            results['allocation'].to_excel(writer, sheet_name='Allocation_Results', index=False)

            # Reports
            for report_name, report_df in results['reports'].items():
                if isinstance(report_df, pd.DataFrame):
                    report_df.to_excel(writer, sheet_name=report_name.replace('_', ' ').title())

            # Suite utilization summary
            allocation_df = results['allocation']
            suite_summary = allocation_df[allocation_df['Suite'] != 'UNMET_DEMAND'].groupby(['Suite', 'Year']).agg({
                'Suite_Batch_Utilization_%': 'max',
                'Allocated_kg': 'sum',
                'Batches_Used': 'sum'
            }).reset_index()
            suite_summary.to_excel(writer, sheet_name='Suite_Summary', index=False)

            # Product summary
            product_summary = allocation_df.groupby(['Product', 'Year']).agg({
                'Allocated_kg': 'sum',
                'Process': 'first'
            }).reset_index()
            product_summary.to_excel(writer, sheet_name='Product_Summary', index=False)

        print(f"📋 Results exported to: {filename}")

def main():
    """Main execution function"""
    print("🏭 Capacity Planning Optimizer")
    print("=" * 50)

    # Initialize optimizer
    optimizer = CapacityPlanningOptimizer()

    # Load demand data (you'll need to specify your file path)
    demand_file = 'demand_data.xlsx'  # Replace with your actual file path

    try:
        demand_df = optimizer.load_demand_data(demand_file)

        if demand_df is not None:
            print(f"✅ Loaded demand data: {demand_df.shape[0]} products")
            print(f"📅 Years covered: {[col for col in demand_df.columns if isinstance(col, (int, float))]}")

            # Run optimization
            print("\n🔄 Running capacity optimization...")
            allocation_df = optimizer.optimize_allocation_by_year_shared_capacity(demand_df)

            # Generate reports
            print("\n📊 Generating comprehensive reports...")
            reports = optimizer.generate_comprehensive_report(allocation_df)

            # Compile results
            results = {
                'allocation': allocation_df,
                'reports': reports
            }

            # Create visualizations
            print("\n🎨 Creating visualizations...")
            optimizer.create_visualizations(results)

            # Export to Excel
            print("\n💾 Exporting results...")
            optimizer.export_results_to_excel(results)

            # Print summary
            print("\n" + "=" * 50)
            print("📈 OPTIMIZATION SUMMARY")
            print("=" * 50)

            yearly_summary = reports['yearly_summary']
            for year in yearly_summary.index:
                fulfillment = yearly_summary.loc[year, 'Fulfillment_%']
                total_demand = yearly_summary.loc[year, 'Total_Demand_kg']
                unmet = yearly_summary.loc[year, 'Unmet_Demand_kg']

                print(f"Year {year}: {fulfillment:.1f}% fulfillment ({total_demand:.0f} kg total, {unmet:.0f} kg unmet)")

            # Top capacity gaps
            if not reports['capacity_gaps'].empty:
                print("\n🚨 TOP CAPACITY GAPS:")
                top_gaps = reports['capacity_gaps'].nlargest(5, 'Unmet_Demand_kg')
                for _, gap in top_gaps.iterrows():
                    print(f"  {gap['Product']} ({gap['Year']}): {gap['Unmet_Demand_kg']:.0f} kg - {gap['Process']}")

            print(f"\n✅ Analysis complete! Check the generated files for detailed results.")

        else:
            print("❌ Failed to load demand data. Please check the file path and format.")

    except FileNotFoundError:
        print(f"❌ File not found: {demand_file}")
        print("Please ensure the demand data file exists and update the file path in the script.")
    except Exception as e:
        print(f"❌ Error during execution: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

🏭 Capacity Planning Optimizer
✅ Loaded demand data: 12 products
📅 Years covered: [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036]

🔄 Running capacity optimization...

Processing Year 2026 with 1 products
  Processing Suite: PW-S1 (Fed-Batch) - Batches available: 22
    ADL-018: Allocated 15.4 kg (22 batches, 100.0% product util)
  Processing Suite: PW-S2 (Fed-Batch) - Batches available: 22
    ADL-018: Allocated 0.6 kg (1 batches, 0.3% product util)
  Processing Suite: Pipan-S1 (Fed-Batch) - Batches available: 22
  Processing Suite: Pipan-S2 (Perfusion) - Batches available: 11
  Processing Suite: PW-S3 (Perfusion) - Batches available: 11
  Processing Suite: Pipan-S6 (Fed-Batch) - Batches available: 22
  Processing Suite: Pipan-S7 (Fed-Batch) - Batches available: 22
  Processing Suite: Pipan-S3 (Fed-Batch) - Batches available: 22
  Processing Suite: Pipan-S4 (Fed-Batch) - Batches available: 22
  Processing Suite: Pipan-S5 (Fed-Batch) - Batches available: 22
  Processin

# New Section